<a href="https://colab.research.google.com/github/BeePandey/Birendra_Info5731_Spring2021/blob/main/In_class_exercise_08_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 3/30/2021)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [3]:
# Write your code here

!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor
import nltk
nltk.download('stopwords')

  Using cached https://files.pythonhosted.org/packages/21/d2/8b0def84a53c88d0eb27c67b05269fbd16ad68df8c78849e7b5d65e6aec3/pycld2-0.41.tar.gz
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834303 sha256=d5f4538144ed1d2de7352874752504c90a3e5a556301f2ff5362864957cf39e3
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
Successfully built pycld2
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
%%bash
polyglot download sentiment2.en


[polyglot_data] Downloading package sentiment2.en to
[polyglot_data]     /root/polyglot_data...


In [5]:

import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/BeePandey/Birendra_Info5731_Spring2021/main/TrainingandEvaluationWest3.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [6]:
from nltk.corpus import stopwords
from textblob import TextBlob
df['Review'] = df['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Review'] = df['Review'].str.replace('[^\w\s]','')
stop = stopwords.words('english')
df['Review'] = df['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['Review'] = df['Review'].apply(lambda x: str(TextBlob(x).correct()))

In [8]:

from polyglot.text import Text
is_sentimental = []
for line in df['Review']:
  text = Text(line)
  for w in text.words:
    if w.polarity != 0:
      is_sentimental.append(w)
is_sentimental

['brilliance',
 'grotesque',
 'hard',
 'mesmerizing',
 'tragic',
 'funny',
 'rollercoaster',
 'proper',
 'best',
 'isolated',
 'sorry',
 'encourage',
 'better',
 'respect',
 'like',
 'isolated',
 'masterpiece',
 'best',
 'best',
 'best',
 'willingly',
 'remarkable',
 'disturbing',
 'violent',
 'proving',
 'corruption',
 'rich',
 'talented',
 'believable',
 'shake',
 'bitter',
 'comforting',
 'perfect',
 'fearless',
 'stunning',
 'impossible',
 'dark',
 'joke',
 'stunned',
 'accomplished',
 'impossible',
 'exceeds',
 'dark',
 'crime',
 'like',
 'falling',
 'joke',
 'happy',
 'ignore',
 'complaints',
 'pernicious',
 'embarrassing',
 'better',
 'criminal',
 'dark',
 'joke',
 'dark',
 'fall',
 'love',
 'bad',
 'romantic',
 'scars',
 'amazing',
 'best',
 'breathtaking',
 'uncomfortable',
 'like',
 'horrible',
 'great',
 'inspiration',
 'honest',
 'violent',
 'disturbing',
 'violent',
 'controversy',
 'great',
 'best',
 'hate',
 'madness',
 'enjoy',
 'masterpiece',
 'amazing',
 'enjoy',
 'jo

In [33]:
from collections import Counter
frequencies = Counter(is_sentimental)
isrank_df = pd.DataFrame(list(frequencies.most_common()), columns=['Words', 'Frequencies'])
isrank_df.index = list(range(1, len(frequencies.most_common())+1))
isrank_df



,Words,Frequencies
1,joke,135
2,like,61
3,good,53
4,best,40
5,dark,33
...,...,...
467,cry,1
468,marvelous,1
469,tolerable,1
470,pleasantly,1


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [35]:
# Write your code here

def Encoder(senti):
  sentiment_encoded = 0
  if senti == 'Positive':
    sentiment_encoded = 1
  elif senti == 'Negative':
    sentiment_encoded = 2
  return sentiment_encoded


In [36]:
isactual_sentiment = []
for senti in df['Sentimental Analysis']:
  isactual_sentiment.append(Encoder(senti))

Sentimental Analysis using Text Blob

In [38]:
from textblob import TextBlob
textblob_sentiments = []
for line in df['Review']:
  polarity = TextBlob(line).sentiment.polarity
  if polarity > 0:
    textblob_sentiments.append(1)
  elif polarity < 0:
    textblob_sentiments.append(2)
  elif polarity == 0.0:
    textblob_sentiments.append(0)
text_blob_df = pd.DataFrame(list(zip(df['Review'], isactual_sentiment, textblob_sentiments)), columns = ['Reviews', 'Actual', 'Predicted'])
text_blob_df

,Reviews,Actual,Predicted
0,every movie comes truly makes impact joaquins ...,1,2
1,movie felt alone isolated truly relate underst...,1,0
2,truly masterpiece best hollywood film 2019 one...,1,1
3,joaquin phoenix gives tour de force performanc...,0,2
4,time moves anticipated like end falling short ...,2,1
...,...,...,...
103,truly masterpiece best hollywood film 2019 one...,1,1
104,acting cinematography sound design script phen...,1,2
105,plan watching would suggest shipping last 20 3...,2,2
106,want title make wouldn know based comicbook ch...,1,1


In [41]:
from sklearn.metrics import f1_score, accuracy_score
istext_blob_accuracy = accuracy_score(text_blob_df['Actual'], text_blob_df['Predicted'])*100
istext_blob_f1_score = f1_score(text_blob_df['Actual'], text_blob_df['Predicted'], average = 'macro')
print("Text Blob Accuracy is {0} and F1 Score is {1}".format(istext_blob_accuracy, istext_blob_f1_score))

Text Blob Accuracy is 56.481481481481474 and F1 Score is 0.399400871459695


Sentimental Analysis using Vader

In [42]:

import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [45]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
vader_sentiments = []
for line in df['Review']:
  polarity = vader.polarity_scores(line)['compound']
  if polarity > 0:
    vader_sentiments.append(1)
  elif polarity < 0:
    vader_sentiments.append(2)
  elif polarity == 0.0:
    vader_sentiments.append(0)
vader_df = pd.DataFrame(list(zip(df['Review'], isactual_sentiment, vader_sentiments)), columns = ['Reviews', 'Actual', 'Predicted'])
vader_df

,Reviews,Actual,Predicted
0,every movie comes truly makes impact joaquins ...,1,1
1,movie felt alone isolated truly relate underst...,1,1
2,truly masterpiece best hollywood film 2019 one...,1,1
3,joaquin phoenix gives tour de force performanc...,0,1
4,time moves anticipated like end falling short ...,2,1
...,...,...,...
103,truly masterpiece best hollywood film 2019 one...,1,1
104,acting cinematography sound design script phen...,1,1
105,plan watching would suggest shipping last 20 3...,2,1
106,want title make wouldn know based comicbook ch...,1,2


In [47]:

from sklearn.metrics import f1_score, accuracy_score
isvader_accuracy = accuracy_score(vader_df['Actual'], vader_df['Predicted'])*100
isvader_f1_score = f1_score(vader_df['Actual'], vader_df['Predicted'], average = 'macro')
print("Vader Accuracy is {0} and F1 Score is {1}".format(isvader_accuracy, isvader_f1_score))

Vader Accuracy is 58.333333333333336 and F1 Score is 0.3673289183222958



Sentimenatl Analysis using SVM

In [49]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:

TFid_vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)


In [51]:

Train_X, Test_X, Train_Y, Test_Y = train_test_split(df['Review'],df['Sentimental Analysis'],test_size=0.33)
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
Train_X_vectors = TFid_vectorizer.fit_transform(Train_X)
Test_X_vectors = TFid_vectorizer.transform(Test_X)

In [53]:
from sklearn import svm
from sklearn.metrics import classification_report
issvm_model = svm.SVC(kernel='linear')
issvm_model.fit(Train_X_vectors, Train_Y)
predicted = issvm_model.predict(Test_X_vectors)
report = classification_report(Test_Y, predicted, output_dict=True)
report

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'f1-score': 0.4761904761904762,
  'precision': 0.7142857142857143,
  'recall': 0.35714285714285715,
  'support': 14},
 '1': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 4},
 '2': {'f1-score': 0.6808510638297872,
  'precision': 0.5517241379310345,
  'recall': 0.8888888888888888,
  'support': 18},
 'accuracy': 0.5833333333333334,
 'macro avg': {'f1-score': 0.3856805133400878,
  'precision': 0.4220032840722496,
  'recall': 0.41534391534391535,
  'support': 36},
 'weighted avg': {'f1-score': 0.5256107171000788,
  'precision': 0.553639846743295,
  'recall': 0.5833333333333334,
  'support': 36}}

In [ ]:
SVM has the highest F1- score and the Text Blob has the second highest F1 score and then Vader. SVM has better accuracy of polarities when compared to TextBlob and Vader and among TextBlob and Vader, Textblob is better